In [2]:
import numpy as np
from tensorflow import keras
from keras.utils import to_categorical

Using TensorFlow backend.


## Task 1

In [4]:
def task1(textfile, window_size, stride):
    # open the file as read, read text, and close file
    file = open(textfile, 'r')
    text = file.read()
    file.close()
    print(text[:50])
    
    #  strip all of the new line characters so that we 
    # have one long sequence of characters separated only 
    # by white space
    tokens = text.split()
    print(tokens[:50])
    data = ' '.join(tokens)
    print(data[:50])
    
    # get sequences of characters of length window_size+1
    sequences = []
    for i in range(0, len(data) - (window_size + 1), stride):
        sequence = data[i : i + window_size + 1]
        sequences.append(sequence)
    print('Total Sequences: %d' % len(sequences))
    
    # save sequences 
    data = '\n'.join(sequences)
    file = open('train_data.txt', 'w')
    file.write(data)
    file.close()

In [5]:
task1('beatles.txt', 3, 4)

a day in the life
i read the news today oh boy
abo
['a', 'day', 'in', 'the', 'life', 'i', 'read', 'the', 'news', 'today', 'oh', 'boy', 'about', 'a', 'lucky', 'man', 'who', 'made', 'the', 'grade', 'and', 'though', 'the', 'news', 'was', 'rather', 'sad', 'well', 'i', 'just', 'had', 'to', 'laugh', 'i', 'saw', 'the', 'photograph.', 'he', 'blew', 'his', 'mind', 'out', 'in', 'a', 'car', 'he', 'didn’t', 'notice', 'that', 'the']
a day in the life i read the news today oh boy abo
Total Sequences: 41686


## Task 2

In [34]:
def task2(train_textfile):
    # open the file as read, read text, and close file
    file = open(train_textfile, 'r')
    text = file.read()
    file.close()
    
    # get list of sequences by splitting the text by new line
    lines = text.split('\n')
    
    # get unique characters
    chars = sorted(list(set(text)))
    # get mapping of character to integer values and store in a dictionary
    char_to_i_mapping = dict((c, i) for i, c in enumerate(chars)) 
    # get vocabulary size
    vocab_size = len(char_to_i_mapping)
    print('Vocabulary size: %d' % vocab_size)
    
    # integer encode each sequence of characters using the dictionary mapping
    sequences = []
    for line in lines:
        # integer encode line
        encoded_seq = [char_to_i_mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    
    # now separate the integer encoded sequences into input and output
    sequences = np.array(sequences)
    X = sequences[:,:-1]
    y = sequences[:,-1]
    
    # now one-hot encode each character, meaning each character becomes a vector of length vocab_size with a 1 marked 
    # for the character and 0s elsewhere
    sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
    X = np.array(sequences)
    y = to_categorical(y, num_classes=vocab_size)
    print('X shape: %s and y shape: %s' %(X.shape, y.shape))
    
    return(X, y)

In [36]:
X, y = task2('train_data.txt')

Vocabulary size: 48
X shape: (41686, 3, 48) and y shape: (41686, 48)


## Other